# Análisis de Reglas de Asociación en Partidas de Ajedrez## Descripción del ProyectoEste notebook implementa el algoritmo Apriori para analizar patrones en partidas de ajedrez de lichess.org. Se estudian dos modalidades de juego: **ajedrez relámpago (600+0)** y **ajedrez bala (60+0)**.### Objetivos1. **Implementación del Algoritmo Apriori**: Aplicación práctica para descubrir reglas de asociación2. **Análisis de Patrones de Juego**: Identificación de relaciones entre variables categóricas3. **Verificación de Hipótesis Específicas**: Validación estadística de reglas propuestas4. **Comparación de Modalidades**: Análisis diferencial entre ritmos de juego### Metodología- **Preprocesamiento**: Limpieza y categorización de variables- **Extracción de reglas**: Aplicación de Apriori con parámetros apropiados- **Métricas de evaluación**: Cálculo de soporte, confianza y lift- **Comparación de resultados**: Visualización de resultados

In [17]:
# Instalación de librerías necesarias!pip install mlxtend# Importación de librerías necesariasimport pandas as pdimport numpy as npimport matplotlib.pyplot as pltimport seaborn as snsfrom mlxtend.frequent_patterns import apriori, association_rulesfrom mlxtend.preprocessing import TransactionEncoderimport warningswarnings.filterwarnings('ignore')# Configuración de pandaspd.set_option('display.max_columns', None)pd.set_option('display.width', None)pd.set_option('display.max_colwidth', None)# Configuración de gráficosplt.style.use('default')sns.set_palette('husl')print('=' * 80)print('    ANÁLISIS DE REGLAS DE ASOCIACIÓN EN PARTIDAS DE AJEDREZ')print('    Algoritmo Apriori - Dataset lichess.org 2013')print('=' * 80)print('\nLibrerías cargadas')

    ANÁLISIS DE REGLAS DE ASOCIACIÓN EN PARTIDAS DE AJEDREZ
    Algoritmo Apriori - Dataset lichess.org 2013

✓ Entorno de análisis configurado correctamente
✓ Librerías especializadas cargadas
✓ Configuración de visualización establecida


## 1. Carga y Exploración Inicial del Dataset### Características del DatasetEl dataset `lichess_games_2013-01.csv` contiene información sobre partidas de ajedrez del portal lichess.org, incluyendo:- **Información de jugadores**: Nombres y puntuaciones Elo- **Características de partida**: Resultado, control de tiempo, número de movimientos- **Información técnica**: Código ECO, apertura, tipo de finalización### Proceso de ExploraciónSe realizará un análisis exploratorio  de los datos.

In [18]:
# Carga del dataset principalprint('Cargando datos...')df = pd.read_csv('lichess_games_2013-01.csv')# Análisis del datasetprint(f'\n INFORMACIÓN DIMENSIONAL')print(f'Dimensiones del dataset: {df.shape[0]:,} filas × {df.shape[1]} columnas')print(f'Memoria utilizada: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB')# Información estructuralprint(f'\n INFORMACIÓN ESTRUCTURAL')print(df.info())# Valores faltantesprint(f'\n INTEGRIDAD DE DATOS')print('Valores faltantes por columna:')missing_data = df.isnull().sum()for col, missing in missing_data.items():    if missing > 0:        percentage = (missing / len(df)) * 100        print(f'  {col}: {missing:,} ({percentage:.2f}%)')    else:        print(f'  {col}: Sin valores faltantes')print('\nDataset cargado')

Iniciando carga del dataset...

 INFORMACIÓN DIMENSIONAL
Dimensiones del dataset: 121,332 filas × 11 columnas
Memoria utilizada: 66.91 MB

 INFORMACIÓN ESTRUCTURAL
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121332 entries, 0 to 121331
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   Site         121332 non-null  object
 1   White        121332 non-null  object
 2   Black        121332 non-null  object
 3   Result       121332 non-null  object
 4   WhiteElo     121332 non-null  object
 5   BlackElo     121332 non-null  object
 6   TimeControl  121332 non-null  object
 7   ECO          121332 non-null  object
 8   Opening      121332 non-null  object
 9   Termination  121332 non-null  object
 10  MovesCount   121332 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 10.2+ MB
None

 INTEGRIDAD DE DATOS
Valores faltantes por columna:
  Site: Sin valores faltantes ✓
  White: Sin valores faltantes ✓
  Bl

## 2. Preprocesamiento y Categorización de Variables### Tratamiento de Valores EspecialesSe realizará la limpieza de datos siguiendo las especificaciones del proyecto:- Conversión de valores "?" en Elo a 900 puntos- Categorización de Elo según rangos estándar- Categorización de número de movimientos por duración### Sistema de Categorización**Categorías de Elo:**- Principiante: 0-1199- Intermedio: 1200-1599- Avanzado: 1600-1999- Experto: 2000-2399- Maestro: 2400-2799- Gran Maestro: 2800+**Categorías de Duración:**- Corta: <20 movimientos- Media: 20-39 movimientos- Larga: 40-59 movimientos- Muy larga: 60+ movimientos

In [19]:
# Creación de copia para procesamientoprint(' INICIANDO PREPROCESAMIENTO DE DATOS')df_processed = df.copy()# Tratamiento de valores especiales en Eloprint('\n1. Tratamiento de valores Elo...')print(f'   Valores "?" en WhiteElo: {(df_processed["WhiteElo"] == "?").sum():,}')print(f'   Valores "?" en BlackElo: {(df_processed["BlackElo"] == "?").sum():,}')# Reemplazar "?" por 900 según especificacionesdf_processed['WhiteElo'] = df_processed['WhiteElo'].replace('?', '900')df_processed['BlackElo'] = df_processed['BlackElo'].replace('?', '900')df_processed['WhiteElo'] = pd.to_numeric(df_processed['WhiteElo'], errors='coerce')df_processed['BlackElo'] = pd.to_numeric(df_processed['BlackElo'], errors='coerce')# Imputación de valores faltantesdf_processed['WhiteElo'].fillna(900, inplace=True)df_processed['BlackElo'].fillna(900, inplace=True)print('   Conversión de Elo realizada ("?" → 900)')print(f'   Rango WhiteElo: {df_processed["WhiteElo"].min():.0f} - {df_processed["WhiteElo"].max():.0f}')print(f'   Rango BlackElo: {df_processed["BlackElo"].min():.0f} - {df_processed["BlackElo"].max():.0f}')

 INICIANDO PREPROCESAMIENTO DE DATOS

1. Tratamiento de valores Elo...
   Valores "?" en WhiteElo: 78
   Valores "?" en BlackElo: 140
   ✓ Conversión de Elo completada ("?" → 900)
   Rango WhiteElo: 782 - 2403
   Rango BlackElo: 789 - 2386


In [20]:
# Funciones de categorizacióndef categorizar_elo(elo):    """Categoriza el Elo según rangos estándar internacionales de ajedrez"""    if elo < 1200:        return "Principiante"    elif elo < 1600:        return "Intermedio"    elif elo < 2000:        return "Avanzado"    elif elo < 2400:        return "Experto"    elif elo < 2800:        return "Maestro"    else:        return "Gran Maestro"def categorizar_movimientos(moves):    """Categoriza la duración de partidas según número de movimientos"""    if moves < 20:        return "Corta"    elif moves < 40:        return "Media"    elif moves < 60:        return "Larga"    else:        return "Muy larga"# Aplicación de categorizacionesprint('\n2. Categorizando variables...')df_processed['WhiteElo_Cat'] = df_processed['WhiteElo'].apply(categorizar_elo)df_processed['BlackElo_Cat'] = df_processed['BlackElo'].apply(categorizar_elo)df_processed['MovesCount_Cat'] = df_processed['MovesCount'].apply(categorizar_movimientos)print('   Categorización de Elo aplicada')print('   Categorización de movimientos aplicada')# Distribucionesprint(f'\n📊 ANÁLISIS DE DISTRIBUCIONES CATEGÓRICAS')print('\nDistribución WhiteElo_Cat:')white_elo_dist = df_processed['WhiteElo_Cat'].value_counts()print(white_elo_dist)print('\nDistribución MovesCount_Cat:')moves_dist = df_processed['MovesCount_Cat'].value_counts()print(moves_dist)


2. Aplicando sistema de categorización...
   ✓ Categorización de Elo completada
   ✓ Categorización de movimientos completada

📊 ANÁLISIS DE DISTRIBUCIONES CATEGÓRICAS

Distribución WhiteElo_Cat:
WhiteElo_Cat
Intermedio      57571
Avanzado        57540
Experto          3550
Principiante     2670
Maestro             1
Name: count, dtype: int64

Distribución MovesCount_Cat:
MovesCount_Cat
Media        61374
Larga        29137
Corta        21750
Muy larga     9071
Name: count, dtype: int64


In [21]:
# Variables derivadasprint('\n3. Calculando variables...')# Diferencia absoluta de Elodf_processed['EloDiff'] = abs(df_processed['WhiteElo'] - df_processed['BlackElo'])# Diferencia en categorías de Elodef diferencia_categorias_elo(white_elo, black_elo):    """Calcula la diferencia en categorías entre jugadores"""    categorias = [0, 1200, 1600, 2000, 2400, 2800, float('inf')]    cat_names = ['Principiante', 'Intermedio', 'Avanzado', 'Experto', 'Maestro', 'Gran Maestro']    # Determinar categoría de cada jugador    white_cat_idx = next(i for i, threshold in enumerate(categorias[1:]) if white_elo < threshold)    black_cat_idx = next(i for i, threshold in enumerate(categorias[1:]) if black_elo < threshold)    return abs(white_cat_idx - black_cat_idx)# Aplicar función de diferencia categóricadf_processed['EloDiff_Cat'] = df_processed.apply(    lambda row: diferencia_categorias_elo(row['WhiteElo'], row['BlackElo']), axis=1)# Jugador con mayor Elodef jugador_mas_fuerte(row):    """Identifica qué jugador tiene mayor Elo"""    if row['WhiteElo'] > row['BlackElo']:        return 'Blanco'    elif row['BlackElo'] > row['WhiteElo']:        return 'Negro'    else:        return 'Empate'df_processed['JugadorMasFuerte'] = df_processed.apply(jugador_mas_fuerte, axis=1)print('   Diferencia de Elo calculada')print('   Diferencia categórica calculada')print('   Jugador más fuerte identificado')# Variables creadasprint(f'\n RESUMEN DE VARIABLES DERIVADAS')print(f'{df_processed.shape[0]:,} filas × {df_processed.shape[1]} columnas')print(f'\nNuevas variables creadas:')print('- WhiteElo_Cat, BlackElo_Cat: Categorías de Elo')print('- MovesCount_Cat: Categorías de duración')print('- EloDiff: Diferencia absoluta de Elo')print('- EloDiff_Cat: Diferencia en categorías')print('- JugadorMasFuerte: Identificación de jugador dominante')


3. Generando variables derivadas...
   ✓ Diferencia de Elo calculada
   ✓ Diferencia categórica calculada
   ✓ Jugador más fuerte identificado

 RESUMEN DE VARIABLES DERIVADAS
Dataset procesado: 121,332 filas × 17 columnas

Nuevas variables creadas:
- WhiteElo_Cat, BlackElo_Cat: Categorías de Elo
- MovesCount_Cat: Categorías de duración
- EloDiff: Diferencia absoluta de Elo
- EloDiff_Cat: Diferencia en categorías
- JugadorMasFuerte: Identificación de jugador dominante


## 3. Análisis del Subconjunto Ajedrez Relámpago (600+0)### Características del Ajedrez RelámpagoEl ajedrez relámpago con control de tiempo "600+0" (10 minutos sin incremento) representa un formato equilibrado entre:- **Tiempo suficiente**: Para desarrollar estrategias- **Presión temporal**: Que añade un presión de tiempo- **Popularidad**: Una de las modalidades más jugadas en plataformas online### Objetivos del Análisis1. **Caracterización del subconjunto**: Análisis descriptivo de las partidas2. **Aplicación de Apriori**: Descubrimiento de patrones frecuentes3. **Extracción de reglas**: Identificación de reglas de asociación significativas4. **Verificación de hipótesis**: Validación de reglas específicas propuestas

In [22]:
# Subconjunto ajedrez relámpagoprint('ANÁLISIS DE AJEDREZ RELÁMPAGO (600+0)')print('=' * 60)# Filtrar partidas con TimeControl='600+0'df_600 = df_processed[df_processed['TimeControl'] == '600+0'].copy()# Análisis de disponibilidad de datosif len(df_600) == 0:    print('  No se encontraron partidas con TimeControl="600+0"')    print('\n🔍 Analizando controles de tiempo disponibles:')    tc_counts = df_processed['TimeControl'].value_counts().head(10)    print(tc_counts)    # Seleccionar el TimeControl más representativo    most_common_tc = tc_counts.index[0]    print(f'\n Usando el TimeControl más común: {most_common_tc}')    df_600 = df_processed[df_processed['TimeControl'] == most_common_tc].copy()else:    most_common_tc = '600+0'# Estadísticas descriptivas del subconjuntoprint(f'\n ESTADÍSTICAS RELÁMPAGO')print(f'Control de tiempo: {most_common_tc}')print(f'Número de partidas: {len(df_600):,}')print(f'Porcentaje del total: {len(df_600)/len(df_processed)*100:.2f}%')if len(df_600) > 0:    print(f'\n DISTRIBUCIÓN DE RESULTADOS')    result_dist = df_600['Result'].value_counts()    for result, count in result_dist.items():        percentage = (count / len(df_600)) * 100        print(f'  {result}: {count:,} partidas ({percentage:.1f}%)')    print(f'\nESTADÍSTICAS PRINCIPALES')    print(f'Elo medio blancas: {df_600["WhiteElo"].mean():.0f}')    print(f'Elo medio negras: {df_600["BlackElo"].mean():.0f}')    print(f'Movimientos medio: {df_600["MovesCount"].mean():.1f}')    print(f'Diferencia Elo media: {df_600["EloDiff"].mean():.0f}')else:    print(' Datos insuficientes')

ANÁLISIS DE AJEDREZ RELÁMPAGO (600+0)

 ESTADÍSTICAS DEL SUBCONJUNTO RELÁMPAGO
Control de tiempo analizado: 600+0
Número de partidas: 2,452
Porcentaje del dataset total: 2.02%

 DISTRIBUCIÓN DE RESULTADOS
  1-0: 1,206 partidas (49.2%)
  0-1: 1,174 partidas (47.9%)
  1/2-1/2: 72 partidas (2.9%)

⚡ CARACTERÍSTICAS TEMPORALES
Elo promedio blancas: 1553
Elo promedio negras: 1552
Movimientos promedio: 32.9
Diferencia Elo promedio: 145


## 4. Implementación del Algoritmo Apriori### Fundamentos TeóricosEl algoritmo Apriori es un método fundamental en minería de datos para descubrir reglas de asociación. Utiliza el principio de que todos los subconjuntos de un conjunto frecuente también son frecuentes.### Métricas de Evaluación- **Soporte**: Frecuencia de aparición del conjunto de ítems- **Confianza**: Probabilidad condicional de ocurrencia- **Lift**: Medida de independencia estadística entre antecedente y consecuente### Preparación de DatosLa implementación requiere transformar los datos categóricos en formato de transacciones binarias.

In [23]:
# Algoritmo Aprioridef preparar_datos_apriori(df_subset):    """Prepara los datos en formato transaccional para Apriori"""    print(' Preparando datos para algoritmo Apriori...')    # Seleccionar variables categóricas relevantes    cols_categoricas = [        'Result', 'WhiteElo_Cat', 'BlackElo_Cat', 'MovesCount_Cat',        'ECO', 'Termination', 'JugadorMasFuerte'    ]    # Filtrar columnas existentes    cols_disponibles = [col for col in cols_categoricas if col in df_subset.columns]    print(f'   Variables disponibles: {len(cols_disponibles)}')    # Crear transacciones    transacciones = []    for _, row in df_subset.iterrows():        transaccion = []        for col in cols_disponibles:            if pd.notna(row[col]):                # Crear etiqueta descriptiva                if col == 'ECO' and len(df_subset) > 1000:  # Limitar ECO para datasets grandes                    continue                transaccion.append(f'{col}_{row[col]}')        transacciones.append(transaccion)    print(f'   Transacciones creadas: {len(transacciones):,}')    print(f'   Ítems por transacción: {np.mean([len(t) for t in transacciones]):.1f}')    return transaccionesdef ejecutar_apriori(df_subset, nombre_subset, min_support=0.01):    """Ejecuta el algoritmo Apriori y extrae reglas de asociación"""    print(f'\n EJECUTANDO APRIORI - {nombre_subset.upper()}')    print('=' * 70)    if len(df_subset) == 0:        print(' Datos insuficientes')        return pd.DataFrame(), pd.DataFrame()    # Preparar datos    transacciones = preparar_datos_apriori(df_subset)    # Codificación binaria    te = TransactionEncoder()    te_ary = te.fit(transacciones).transform(transacciones)    df_encoded = pd.DataFrame(te_ary, columns=te.columns_)    print(f'\n Matriz binaria: {df_encoded.shape[0]} × {df_encoded.shape[1]}')    # Aplicar Apriori para encontrar conjuntos frecuentes    print(f'\n  Aplicando Apriori (soporte: {min_support})')    frequent_itemsets = apriori(df_encoded, min_support=min_support, use_colnames=True)    if len(frequent_itemsets) == 0:        print(f'  Sin conjuntos frecuentes con soporte >= {min_support}')        print('   Reduciendo umbral de soporte')        frequent_itemsets = apriori(df_encoded, min_support=min_support/2, use_colnames=True)    print(f'   Conjuntos frecuentes encontrados: {len(frequent_itemsets)}')    if len(frequent_itemsets) == 0:        return pd.DataFrame(), pd.DataFrame()    # Generar reglas de asociación    print('\n Extrayendo reglas...')    try:        rules = association_rules(frequent_itemsets, metric='confidence', min_threshold=0.1)        if len(rules) > 0:            # Ordenar por lift descendente            rules_sorted = rules.sort_values('lift', ascending=False)            print(f'   Reglas generadas: {len(rules)}')            print(f'\n TOP 10 REGLAS MÁS SIGNIFICATIVAS:')            print('-' * 120)            for i, (_, rule) in enumerate(rules_sorted.head(10).iterrows(), 1):                antecedents = ', '.join(list(rule['antecedents']))                consequents = ', '.join(list(rule['consequents']))                print(f'{i:2d}. {antecedents} → {consequents}')                print(f'    Soporte: {rule["support"]:.4f} | '                      f'Confianza: {rule["confidence"]:.4f} | '                      f'Lift: {rule["lift"]:.4f}')                print()            return frequent_itemsets, rules_sorted        else:            print('     Sin reglas con estos umbrales')            return frequent_itemsets, pd.DataFrame()    except Exception as e:        print(f'    Error en extracción de reglas: {str(e)}')        return frequent_itemsets, pd.DataFrame()# Ejecutar Apriori para el subconjunto relámpagoif len(df_600) > 0:    frequent_600, rules_600 = ejecutar_apriori(df_600, 'Ajedrez Relámpago')else:    print(' Subconjunto vacío')    frequent_600, rules_600 = pd.DataFrame(), pd.DataFrame()


 EJECUTANDO APRIORI - AJEDREZ RELÁMPAGO
 Preparando datos para algoritmo Apriori...
   Variables categóricas disponibles: 7
   Transacciones generadas: 2,452
   Ítems promedio por transacción: 6.0

 Matriz de codificación: 2452 × 20

  Aplicando Apriori (soporte mínimo: 0.01)
   ✓ Conjuntos frecuentes encontrados: 873

 Generando reglas de asociación...
   ✓ Reglas generadas: 9346

 TOP 10 REGLAS MÁS SIGNIFICATIVAS:
------------------------------------------------------------------------------------------------------------------------
 1. MovesCount_Cat_Muy larga, Termination_Normal → Result_1/2-1/2
    Soporte: 0.0114 | Confianza: 0.2171 | Lift: 7.3919

 2. Result_1/2-1/2 → MovesCount_Cat_Muy larga, Termination_Normal
    Soporte: 0.0114 | Confianza: 0.3889 | Lift: 7.3919

 3. Result_1/2-1/2 → MovesCount_Cat_Muy larga
    Soporte: 0.0143 | Confianza: 0.4861 | Lift: 7.0114

 4. MovesCount_Cat_Muy larga → Result_1/2-1/2
    Soporte: 0.0143 | Confianza: 0.2059 | Lift: 7.0114

 5. MovesC

## 5. Verificación de Hipótesis Específicas### Metodología de VerificaciónSe implementará un sistema de verificación estadística para evaluar las siguientes hipótesis:1. **H1**: Diferencia de Elo ≥ 1 categoría → Victoria del jugador más fuerte2. **H2**: Diferencia de Elo ≥ 2 categorías → Victoria del jugador más fuerte3. **H3**: Ambos jugadores Gran Maestros → Resultado en tablas4. **H4**: Ambos jugadores Gran Maestros → Victoria de blancas5. **H5**: Jugadores principiantes/intermedios → Victoria de blancas6. **H6**: Terminación Normal/Time forfeit según duración de partida### Métricas de EvaluaciónPara cada hipótesis se calculará:- **Soporte**: Frecuencia absoluta de ocurrencia- **Confianza**: Probabilidad condicional- **Interpretación**: Validez estadística de la regla

In [25]:
# Verificación de hipótesisdef verificar_regla_diferencia_elo(df_subset, diferencia_min, nombre_regla):    """Verifica reglas relacionadas con diferencias de Elo"""    print(f'\n VERIFICACIÓN: {nombre_regla}')    print('-' * 80)    if len(df_subset) == 0:        print(' Sin datos para verificar')        return    # Filtrar casos con diferencia significativa    casos_diferencia = df_subset[df_subset['EloDiff_Cat'] >= diferencia_min]    print(f'Total de partidas: {len(df_subset):,}')    print(f'Partidas con diferencia ≥{diferencia_min} categorías: {len(casos_diferencia):,}')    if len(casos_diferencia) == 0:        print('  Casos insuficientes')        return    # Analizar por color del jugador más fuerte    for color in ['Blanco', 'Negro']:        casos_color = casos_diferencia[casos_diferencia['JugadorMasFuerte'] == color]        if len(casos_color) == 0:            continue        # Determinar resultado esperado        resultado_esperado = '1-0' if color == 'Blanco' else '0-1'        casos_ganados = len(casos_color[casos_color['Result'] == resultado_esperado])        # Calcular métricas        soporte = len(casos_color) / len(df_subset)        confianza = casos_ganados / len(casos_color) if len(casos_color) > 0 else 0        print(f'\n Análisis - Jugador más fuerte con {color.lower()}s:')        print(f'   Casos válidos: {len(casos_color):,}')        print(f'   Victorias esperadas: {casos_ganados:,}')        print(f'   Soporte: {soporte:.4f}')        print(f'   Confianza: {confianza:.4f} ({confianza*100:.1f}%)')        # Interpretación        if confianza >= 0.7:            interpretacion = 'REGLA FUERTE'        elif confianza >= 0.5:            interpretacion = 'REGLA MODERADA'        else:            interpretacion = 'REGLA DÉBIL'        print(f'   Interpretación: {interpretacion}')def verificar_regla_grandes_maestros(df_subset, resultado_esperado, nombre_regla):    """Verifica reglas específicas para Grandes Maestros"""    print(f'\n VERIFICACIÓN: {nombre_regla}')    print('-' * 80)    # Filtrar partidas donde ambos son Grandes Maestros    casos_gm = df_subset[        (df_subset['WhiteElo_Cat'] == 'Gran Maestro') &        (df_subset['BlackElo_Cat'] == 'Gran Maestro')    ]    print(f'Total de partidas: {len(df_subset):,}')    print(f'Partidas GM vs GM: {len(casos_gm):,}')    if len(casos_gm) == 0:        print('  Sin partidas GM vs GM')        return    # Análisis por duración de partida    for categoria in df_subset['MovesCount_Cat'].unique():        casos_cat = casos_gm[casos_gm['MovesCount_Cat'] == categoria]        if len(casos_cat) == 0:            continue        casos_resultado = len(casos_cat[casos_cat['Result'] == resultado_esperado])        soporte = len(casos_cat) / len(df_subset)        confianza = casos_resultado / len(casos_cat) if len(casos_cat) > 0 else 0        print(f'\n Partidas {categoria.lower()}s:')        print(f'   GM vs GM: {len(casos_cat):,}')        print(f'   Resultado "{resultado_esperado}": {casos_resultado:,}')        print(f'   Soporte: {soporte:.4f}')        print(f'   Confianza: {confianza:.4f} ({confianza*100:.1f}%)')# Ejecutar todas las verificaciones de hipótesisprint('\n VERIFICACIÓN SISTEMÁTICA DE HIPÓTESIS')print('=' * 80)if len(df_600) > 0:    # H1 y H2: Diferencias de Elo    verificar_regla_diferencia_elo(df_600, 1, 'H1: Diferencia ≥1 categoría → Victoria jugador fuerte')    verificar_regla_diferencia_elo(df_600, 2, 'H2: Diferencia ≥2 categorías → Victoria jugador fuerte')    # H3 y H4: Grandes Maestros    verificar_regla_grandes_maestros(df_600, '1/2-1/2', 'H3: GM vs GM → Tablas')    verificar_regla_grandes_maestros(df_600, '1-0', 'H4: GM vs GM → Victoria blancas')else:    print(' Sin datos para verificar')


 VERIFICACIÓN SISTEMÁTICA DE HIPÓTESIS

 VERIFICACIÓN: H1: Diferencia ≥1 categoría → Victoria jugador fuerte
--------------------------------------------------------------------------------
Total de partidas analizadas: 2,452
Partidas con diferencia ≥1 categorías: 1,014

 Análisis - Jugador más fuerte con blancos:
   Casos aplicables: 508
   Victorias del más fuerte: 349
   Soporte: 0.2072
   Confianza: 0.6870 (68.7%)
   Interpretación: REGLA MODERADA

 Análisis - Jugador más fuerte con negros:
   Casos aplicables: 506
   Victorias del más fuerte: 347
   Soporte: 0.2064
   Confianza: 0.6858 (68.6%)
   Interpretación: REGLA MODERADA

 VERIFICACIÓN: H2: Diferencia ≥2 categorías → Victoria jugador fuerte
--------------------------------------------------------------------------------
Total de partidas analizadas: 2,452
Partidas con diferencia ≥2 categorías: 31

 Análisis - Jugador más fuerte con blancos:
   Casos aplicables: 16
   Victorias del más fuerte: 15
   Soporte: 0.0065
   Confia